## RDP - Stone Soup Experiments
[Documentation](https://stonesoup.readthedocs.io/en/v1.4/auto_examples/readers/Custom_Pandas_Dataloader.html#sphx-glr-auto-examples-readers-custom-pandas-dataloader-py)

Continuing experiments - following tutorial #76: Probabilistic Data Association.

For this tutorial, I need to extend the test data set from the previous notebook to add a second target. I'll look for a target of opportunity that is observed at the same time as the target considered so far. If possible, a few different cases would be interesting:
1. A straight flight that does not interesect the first flight path
1. A straight flight that DOES intersect the first flight path, but at a different time: 10610889
1. A non-linear flight that does not cross the first flight
1. A non-linear flight that DOES cross the first flight path

In [ ]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from importlib import reload  # Python 3.4+
from typing import Tuple
import itertools
from matplotlib import pyplot as plt
from math import ceil
from ordered_set import OrderedSet

import dateutil
from pymap3d import geodetic2enu

import sys
sys.path.append('C:/Users/ttrinter/git_repo/cspeed/data_common')
sys.path.append('../../..')
import data_functions as dfunc
import visualizations as v
from ttt_ss_funcs import generate_timestamps, ADSBTruthReader, CSVReaderXY, CSVReaderPolar, plot_all, CSVClutterReaderXY, group_plots


from stonesoup.reader import DetectionReader, GroundTruthReader
from stonesoup.base import Property
from stonesoup.types.detection import Detection, Clutter
from stonesoup.plotter import AnimatedPlotterly, Plotter

from stonesoup.base import Property
from stonesoup.buffered_generator import BufferedGenerator
from stonesoup.functions import cart2sphere, sphere2cart
from stonesoup.models.measurement.linear import LinearGaussian
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange
from stonesoup.types.angle import Bearing
from stonesoup.types.detection import Detection
from stonesoup.types.groundtruth import GroundTruthState, GroundTruthPath

# Tracker Imports
from stonesoup.types.state import GaussianState

plot_type = 'static' # or 'animated'

sensor_positions = { 'RDU103': (51.52126391, 5.85862734)}

METERS_in_NM = 1852


## Save/Read  to/from CSV

In [ ]:

from math import pi
data_dir = 'C:/Users/ttrinter/git_repo/Stone-Soup/data'

# Now there are 2 ADSB Files
adsb_data = pd.DataFrame()
adsb_files = [f'{data_dir}/adsb_straight.csv', f'{data_dir}/adsb_straight2.csv']
for file in adsb_files:
    this_data = pd.read_csv(file)
    this_data['timestamp'] = pd.to_datetime(this_data['timestamp'], errors='coerce')
    this_data = this_data.loc[~this_data['timestamp'].isna()]
    this_data['timestamp'] = pd.to_datetime(this_data['timestamp'], errors='coerce')
    this_data['timestamp'] = this_data['timestamp'].dt.tz_localize(None)

    adsb_data = pd.concat([adsb_data, this_data])

rdp_file = f'{data_dir}/rdp_straight.csv'
# rdp_straight['timestamp'] = pd.to_datetime(rdp_straight['timestamp'], errors='coerce')
# rdp_straight['theta_rad'] = np.deg2rad(rdp_straight.theta)
# rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] = rdp_straight.loc[rdp_straight.theta_rad>2*pi, 'theta_rad'] - 2*pi 

# rdp_straight = rdp_straight.loc[~rdp_straight['timestamp'].isna()]
# rdp_straight.to_csv(rdp_file, index=False)
rdp_data = pd.read_csv(rdp_file)
rdp_data['timestamp'] = pd.to_datetime(rdp_data['timestamp'], errors='coerce')
rdp_data['timestamp'] = rdp_data['timestamp'].dt.tz_localize(None)


# Matched Plots
matched_csv1 = f'{data_dir}/rdp_matched.csv'
matched_csv2 = f'{data_dir}/rdp_matched2.csv'
rdp_matched = pd.read_csv(matched_csv1)
rdp_matched['timestamp'] = pd.to_datetime(rdp_matched['timestamp'], errors='coerce')
rdp_matched['timestamp'] = rdp_matched['timestamp'].dt.tz_localize(None)

start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()


clutter_filename = f'{data_dir}/sample_clutter.csv'
clutter_data = pd.read_csv(clutter_filename)

print(f'ADSB: {len(adsb_data)}')
print(f'RDP: {len(rdp_data)}')

In [ ]:
v.scatter_targets(clutter_data)
plt.suptitle('Clutter')

## Matched Data Set
To make things even simpler, I'll grab the set of matched data for this test plane. Then most of the plots should be "true" detections. Let's see how the tracker does with that.

In [ ]:
target_address1 = 10537421 # plane #1
target_address2 = 10610889 # plane #
target_addresses = [target_address1, target_address2]
file_dir = 'C:/Users/ttrinter/OneDrive - cspeed.com (1)/Documents/Data/Travis/2024-07-17'
matched_file = '20240717_Travis_matched_rdp_61.xlsx'
all_matched_data = pd.read_excel(f'{file_dir}/{matched_file}')
matched_data = all_matched_data.loc[(all_matched_data.target_address.isin(target_addresses)) &
                                (all_matched_data.close_enough==True) & 
                                (all_matched_data.timestamp_adsb>=start_time) & 
                                (all_matched_data.timestamp_adsb<=end_time)]
matched_data.head()

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address1, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
matched_plot = v.plot_target_match2(matching=matched_data, 
                                    target_address=target_address2, 
                                    plot_show=True, 
                                    pd_loc='title')  

In [ ]:
# Plotting
fig, ax = plt.subplots()

# Group by the 'category' column and plot each group separately
for tgt, group in rdp_matched.groupby('target_address'):
    ax.scatter(group['x'], group['y'], label=tgt, marker="+")

# Add labels and title
plt.legend()
plt.grid()
plt.title("Matched RDP Plots")

In [ ]:
rdp_matched['x_m'] = rdp_matched.x * METERS_in_NM
rdp_matched['y_m'] = rdp_matched.y * METERS_in_NM
rdp_matched.sort_values('timestamp', inplace=True)

# Track-starts:
track_start1_t = rdp_matched.loc[rdp_matched.target_address==target_address1].iloc[0]['timestamp']
track_start1_x = rdp_matched.loc[rdp_matched.target_address==target_address1].iloc[0]['x_m']
track_start1_y = rdp_matched.loc[rdp_matched.target_address==target_address1].iloc[0]['y_m']

# track_start2_t = rdp_matched.loc[rdp_matched.target_address==target_address2].iloc[0]['timestamp']
# track_start2_x = rdp_matched.loc[rdp_matched.target_address==target_address2].iloc[0]['x_m']
# track_start2_y = rdp_matched.loc[rdp_matched.target_address==target_address2].iloc[0]['y_m']

rdp_matched[['target_address','timestamp', 'rho','theta','x_m','y_m']].head()

In [ ]:
adsb = ADSBTruthReader.multiple_ground_truth_reader(adsb_files)

In [ ]:
plotter = Plotter()
plotter.plot_ground_truths(adsb, 
                        mapping=[0, 2], 
                        markersize = 5,
                        marker = 's', 
                        markerfacecolor = 'none', 
                        alpha = 0.2)
plt.grid()
plt.title("Two Targets of Opportunity")

In [ ]:
# Detections
matched_xy = CSVReaderXY(matched_csv1)
matched_polar = CSVReaderPolar(matched_csv1)

# Cutter
clutter = CSVClutterReaderXY(clutter_filename)

dets = [next(iter(detection[1])) for detection in matched_xy.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)

plot_all(start_time, 
         end_time,
         all_measurements=all_measurements, 
        #  adsb=adsb, 
         plot_type='static')

## From Tutorial #7
This is more or less the sam as tutorial 5, but changing out the Hypothesizer for a PDA hypothesizer. I'm using tutorial 6, rather than 5 to see how it handles the multiple tracks. Maybe it will resolve the weird, extended track issue from #6.

### Probability of detection
For the first time we introduce the possibility that, at any time-step, our sensor receives no detection from the target (i.e. $p_d < 1$).

In [ ]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.kalman import UnscentedKalmanPredictor
from stonesoup.updater.kalman import UnscentedKalmanUpdater

from stonesoup.types.track import Track
from stonesoup.measures import Mahalanobis, Euclidean

from stonesoup.types.array import StateVectors  # For storing state vectors during association
from stonesoup.functions import gm_reduce_single  # For merging states to get posterior estimate
from stonesoup.types.update import GaussianStateUpdate  # To store posterior estimate

from stonesoup.hypothesiser.probability import PDAHypothesiser
from stonesoup.dataassociator.probability import PDA

In [ ]:
# Measurement Model
np.random.seed(42)
default_variance = 50

measurement_model = LinearGaussian(
    ndim_state=4,   # Number of state dimensions (position and velocity in 2D)
    mapping=(0, 2), # Mapping measurement vector index to state index
    noise_covar=np.array([[default_variance, 0 ],  
                          [0, default_variance]])
    )  #Covariance matrix for Gaussian PDF

# Transition Model
q_const = 40
q_x = q_const
q_y = q_const
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

In [ ]:
# predictor = KalmanPredictor(transition_model)
# updater = KalmanUpdater(measurement_model)

predictor = UnscentedKalmanPredictor(transition_model)
updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

prob_det = 0.9
hypothesiser = PDAHypothesiser(predictor=predictor,
                               updater=updater,
                               clutter_spatial_density=0.1,
                               prob_detect=prob_det)

data_associator = PDA(hypothesiser=hypothesiser)

# Clear things out from prior runs
hypothesis = None
post = None

prior1 = GaussianState([[track_start1_x], [1], [track_start1_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)
# prior2 = GaussianState([[track_start2_x], [1], [track_start2_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

# create a prior using the location of the radar
# prior = GaussianState([[0], [q_const], [0], [q_const]], np.diag([default_variance, 0.5, default_variance, 0.5]), timestamp=start_time)

# Loop through the predict, hypothesise, associate and update steps.
# track = Track([prior1])
if "tracks" in globals():
    del tracks

if "track" in globals():
    del(track)

# tracks = {Track([prior1]), Track([prior2])}
# tracks = {Track([prior1])}
track = Track([prior1])

grouped_sec, grouped_pass = group_plots(all_measurements)
for n, measurements in enumerate(grouped_sec):
    this_time = min(measurements, key=lambda meas: meas.timestamp).timestamp
    this_time = this_time.replace(microsecond=0)

    hypotheses = data_associator.associate({track},
                                           measurements,
                                           this_time)

    hypotheses = hypotheses[track]

    # Loop through each hypothesis, creating posterior states for each, and merge to calculate
    # approximation to actual posterior state mean and covariance.
    posterior_states = []
    posterior_state_weights = []
    for hypothesis in hypotheses:
        if not hypothesis:
            posterior_states.append(hypothesis.prediction)
        else:
            posterior_state = updater.update(hypothesis)
            posterior_states.append(posterior_state)
        posterior_state_weights.append(
            hypothesis.probability)

    means = StateVectors([state.state_vector for state in posterior_states])
    covars = np.stack([state.covar for state in posterior_states], axis=2)
    weights = np.asarray(posterior_state_weights)

    # Reduce mixture of states to one posterior estimate Gaussian.
    post_mean, post_covar = gm_reduce_single(means, covars, weights)

    # Add a Gaussian state approximation to the track.
    track.append(GaussianStateUpdate(
        post_mean, post_covar,
        hypotheses,
        hypotheses[0].measurement.timestamp))

## Plot the ground truth and measurements with clutter.

In [ ]:
plot_all(start_time,
         end_time,
         all_measurements=all_measurements, 
        #  adsb=adsb,
         tracks=track,
         plot_type='static')

## Polar Coordinates
Trying again, but changing the process to read rho and theta and, maybe later also radial velocity.

In [ ]:
plot_type='static'
# rdp = RDPReader(rdp_file,
start_time = rdp_matched['timestamp'].min()
end_time = rdp_matched['timestamp'].max()
timestamps = generate_timestamps(start_time, end_time)

# Detections
meas_polar = CSVReaderPolar(matched_csv2)

dets = [next(iter(detection[1])) for detection in meas_polar.detections_gen()]
cluts = [next(iter(detection[1])) for detection in clutter.detections_gen()]

# Combine detections with clutter
all_measurements = dets + cluts
all_measurements.sort(key=lambda obj: obj.timestamp)

### Kalman Filtering Again...
Should be the same from here forward.

In [ ]:
# measurement_model = CartesianToBearingRange(ndim_state=4, 
#                                             mapping=(0,2), 
#                                             noise_covar=np.array([[default_variance, 0 ],
#                                                                   [0, default_variance]]))

predictor = UnscentedKalmanPredictor(transition_model)
updater = UnscentedKalmanUpdater(measurement_model)  # Keep alpha as default = 0.5

prob_det = 0.9
hypothesiser = PDAHypothesiser(predictor=predictor,
                               updater=updater,
                               clutter_spatial_density=0.125,
                               prob_detect=prob_det)

data_associator = PDA(hypothesiser=hypothesiser)

# Clear things out from prior runs
hypothesis = None
post = None

# create a prior using the approximate start of the track
# prior = GaussianState([[track_start_x], [1], [track_start_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

prior1 = GaussianState([[track_start1_x], [1], [track_start1_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)
# prior2 = GaussianState([[track_start2_x], [1], [track_start2_y], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

# create a prior using the location of the radar
# prior = GaussianState([[0], [q_const], [0], [q_const]], np.diag([default_variance, 0.5, default_variance, 0.5]), timestamp=start_time)

# Loop through the predict, hypothesise, associate and update steps.
# del(tracks)
if "tracks" in globals():
    del tracks

# tracks = {Track([prior1]), Track([prior2])}
tracks = {Track([prior1])}
track = Track([prior1])

grouped_sec, grouped_pass = group_plots(all_measurements)
for n, measurements in enumerate(grouped_pass):
    this_time = min(measurements, key=lambda meas: meas.timestamp).timestamp
    this_time = this_time.replace(microsecond=0)
    # print(this_time)

    # # Calculate all hypothesis pairs and associate the elements in the best subset to the tracks.
    # if len(measurements)>0:
    # # for n, measurements in enumerate(dets):
    #     # try: 
    #     hypotheses = data_associator.associate(tracks,
    #                                             measurements,
    #                                             this_time)
    #     for track in tracks:
    #         hypothesis = hypotheses[track]
    #         if hypothesis.measurement:
    #             post = updater.update(hypothesis)
    #             track.append(post)
    #             print('track updated')
    #         else:  # When data associator says no detections are good enough, we'll keep the prediction
    #             track.append(hypothesis.prediction)
    #             print('no update')

    #     # except:
    #     #     print(f'{this_time}: {len(measurements)}: ERROR')
    #     #     continue

    try:
        hypotheses = data_associator.associate({track},
                                            measurements,
                                            this_time)

        hypotheses = hypotheses[track]

        # Loop through each hypothesis, creating posterior states for each, and merge to calculate
        # approximation to actual posterior state mean and covariance.
        posterior_states = []
        posterior_state_weights = []
        for hypothesis in hypotheses:
            if not hypothesis:
                posterior_states.append(hypothesis.prediction)
            else:
                posterior_state = updater.update(hypothesis)
                posterior_states.append(posterior_state)
            posterior_state_weights.append(
                hypothesis.probability)

        means = StateVectors([state.state_vector for state in posterior_states])
        covars = np.stack([state.covar for state in posterior_states], axis=2)
        weights = np.asarray(posterior_state_weights)

        # Reduce mixture of states to one posterior estimate Gaussian.
        post_mean, post_covar = gm_reduce_single(means, covars, weights)

        # Add a Gaussian state approximation to the track.
        track.append(GaussianStateUpdate(
            post_mean, post_covar,
            hypotheses,
            hypotheses[0].measurement.timestamp))
        print(f'Track updated {this_time}')
        
    except:
        print(f'Error at {this_time}')


In [ ]:
plot_all(start_time,
         end_time,
         all_measurements=all_measurements, 
         adsb=adsb,
         tracks=track,
         plot_type='static')

In [ ]:
len(track)